# Day 1: Walk-Forward Validation

## Learning Objectives
- Implement walk-forward optimization
- Compare expanding vs rolling windows
- Build proper time-series cross-validation

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 1: Walk-Forward Validation")

In [ ]:
# ============================================================
# GENERATE DATA WITH REGIME CHANGES
# ============================================================

print("GENERATING DATA WITH REGIME CHANGES")
print("="*60)

np.random.seed(42)
n_days = 2500

# Create returns with regime changes
returns = np.zeros(n_days)

# Regime 1: Low vol, trending (days 0-800)
returns[:800] = np.random.normal(0.0003, 0.012, 800)
for i in range(1, 800):
    returns[i] += 0.1 * returns[i-1]

# Regime 2: High vol, mean-reverting (days 800-1500)
returns[800:1500] = np.random.normal(0.0001, 0.025, 700)
for i in range(801, 1500):
    returns[i] -= 0.05 * returns[i-1]

# Regime 3: Mixed (days 1500-2500)
returns[1500:] = np.random.normal(0.00015, 0.018, 1000)

prices = 100 * np.cumprod(1 + returns)
df = pd.DataFrame({'price': prices, 'returns': returns})

# Features
df['ret_5d'] = df['price'].pct_change(5)
df['ret_20d'] = df['price'].pct_change(20)
df['vol_5d'] = df['returns'].rolling(5).std()
df['vol_20d'] = df['returns'].rolling(20).std()
df['mom_5d'] = df['returns'].rolling(5).sum()
df['mom_20d'] = df['returns'].rolling(20).sum()

df['target'] = (df['returns'].shift(-1) > 0).astype(int)
df = df.dropna()

feature_cols = ['ret_5d', 'ret_20d', 'vol_5d', 'vol_20d', 'mom_5d', 'mom_20d']
print(f"Data shape: {df.shape}")

## Part 1: Walk-Forward Splitter

In [ ]:
# ============================================================
# WALK-FORWARD SPLITTER
# ============================================================

class WalkForwardSplitter:
    """Walk-forward cross-validation for time series."""
    
    def __init__(self, n_splits=5, train_size=500, test_size=100, gap=0, expanding=False):
        """
        Args:
            n_splits: Number of folds
            train_size: Training window size
            test_size: Test window size
            gap: Gap between train and test (avoid lookahead)
            expanding: If True, use expanding window; else rolling
        """
        self.n_splits = n_splits
        self.train_size = train_size
        self.test_size = test_size
        self.gap = gap
        self.expanding = expanding
        
    def split(self, X):
        """Generate train/test indices."""
        n_samples = len(X)
        
        for i in range(self.n_splits):
            if self.expanding:
                # Expanding: start always at 0
                train_start = 0
                train_end = self.train_size + i * self.test_size
            else:
                # Rolling: fixed window size
                train_start = i * self.test_size
                train_end = train_start + self.train_size
            
            test_start = train_end + self.gap
            test_end = test_start + self.test_size
            
            if test_end > n_samples:
                break
                
            train_idx = np.arange(train_start, train_end)
            test_idx = np.arange(test_start, test_end)
            
            yield train_idx, test_idx
            
    def visualize(self, n_samples, ax=None):
        """Visualize the splits."""
        if ax is None:
            fig, ax = plt.subplots(figsize=(14, 4))
        
        for fold, (train_idx, test_idx) in enumerate(self.split(np.zeros(n_samples))):
            ax.barh(fold, len(train_idx), left=train_idx[0], height=0.6, color='steelblue', alpha=0.7, label='Train' if fold==0 else '')
            ax.barh(fold, len(test_idx), left=test_idx[0], height=0.6, color='coral', alpha=0.7, label='Test' if fold==0 else '')
        
        ax.set_xlabel('Time Index')
        ax.set_ylabel('Fold')
        mode = 'Expanding' if self.expanding else 'Rolling'
        ax.set_title(f'Walk-Forward Validation ({mode})', fontweight='bold')
        ax.legend()
        ax.invert_yaxis()
        
        return ax

print("✅ WalkForwardSplitter class defined")

In [ ]:
# ============================================================
# VISUALIZE SPLITTING STRATEGIES
# ============================================================

print("VISUALIZING SPLITTING STRATEGIES")
print("="*60)

n_samples = len(df)

fig, axes = plt.subplots(2, 1, figsize=(14, 6))

# Rolling window
rolling_splitter = WalkForwardSplitter(n_splits=8, train_size=400, test_size=200, expanding=False)
rolling_splitter.visualize(n_samples, ax=axes[0])
axes[0].set_title('Rolling Window Walk-Forward', fontweight='bold')

# Expanding window
expanding_splitter = WalkForwardSplitter(n_splits=8, train_size=400, test_size=200, expanding=True)
expanding_splitter.visualize(n_samples, ax=axes[1])
axes[1].set_title('Expanding Window Walk-Forward', fontweight='bold')

plt.tight_layout()
plt.show()

## Part 2: Walk-Forward Backtesting

In [ ]:
# ============================================================
# WALK-FORWARD BACKTEST
# ============================================================

def walk_forward_backtest(X, y, splitter, model_class, **model_params):
    """
    Run walk-forward backtest.
    
    Returns:
        results: Dict with predictions and metrics per fold
    """
    results = {
        'fold': [],
        'train_acc': [],
        'test_acc': [],
        'predictions': [],
        'actuals': [],
        'test_indices': []
    }
    
    scaler = StandardScaler()
    
    for fold, (train_idx, test_idx) in enumerate(splitter.split(X)):
        # Split data
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        # Scale (fit on train only)
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train model
        model = model_class(**model_params)
        model.fit(X_train_scaled, y_train)
        
        # Predict
        train_pred = model.predict(X_train_scaled)
        test_pred = model.predict(X_test_scaled)
        
        # Store results
        results['fold'].append(fold)
        results['train_acc'].append(accuracy_score(y_train, train_pred))
        results['test_acc'].append(accuracy_score(y_test, test_pred))
        results['predictions'].extend(test_pred)
        results['actuals'].extend(y_test)
        results['test_indices'].extend(test_idx)
        
    return results

print("✅ walk_forward_backtest function defined")

In [ ]:
# ============================================================
# COMPARE ROLLING VS EXPANDING
# ============================================================

print("COMPARING ROLLING VS EXPANDING WINDOWS")
print("="*60)

X = df[feature_cols].values
y = df['target'].values

# Rolling window
rolling_splitter = WalkForwardSplitter(n_splits=10, train_size=500, test_size=150, expanding=False)
rolling_results = walk_forward_backtest(X, y, rolling_splitter, RandomForestClassifier, n_estimators=50, max_depth=5, random_state=42)

# Expanding window
expanding_splitter = WalkForwardSplitter(n_splits=10, train_size=500, test_size=150, expanding=True)
expanding_results = walk_forward_backtest(X, y, expanding_splitter, RandomForestClassifier, n_estimators=50, max_depth=5, random_state=42)

print("\nRolling Window Results:")
for i, (train_acc, test_acc) in enumerate(zip(rolling_results['train_acc'], rolling_results['test_acc'])):
    print(f"  Fold {i}: Train={train_acc:.3f}, Test={test_acc:.3f}")
print(f"  Mean Test Accuracy: {np.mean(rolling_results['test_acc']):.3f}")

print("\nExpanding Window Results:")
for i, (train_acc, test_acc) in enumerate(zip(expanding_results['train_acc'], expanding_results['test_acc'])):
    print(f"  Fold {i}: Train={train_acc:.3f}, Test={test_acc:.3f}")
print(f"  Mean Test Accuracy: {np.mean(expanding_results['test_acc']):.3f}")

In [ ]:
# ============================================================
# VISUALIZE RESULTS
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy over folds
folds = range(len(rolling_results['train_acc']))
axes[0].plot(folds, rolling_results['test_acc'], 'o-', label='Rolling', color='steelblue', linewidth=2)
axes[0].plot(folds, expanding_results['test_acc'], 's-', label='Expanding', color='coral', linewidth=2)
axes[0].axhline(0.5, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlabel('Fold')
axes[0].set_ylabel('Test Accuracy')
axes[0].set_title('Test Accuracy per Fold', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Overfitting check
x = np.arange(len(folds))
width = 0.35
axes[1].bar(x - width/2, rolling_results['train_acc'], width, label='Train', alpha=0.7)
axes[1].bar(x + width/2, rolling_results['test_acc'], width, label='Test', alpha=0.7)
axes[1].set_xlabel('Fold')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Train vs Test Accuracy (Rolling)', fontweight='bold')
axes[1].legend()
axes[1].set_xticks(x)

plt.tight_layout()
plt.show()

print("\n💡 KEY INSIGHT:")
print("   - Rolling adapts better to regime changes")
print("   - Expanding may capture more patterns but include stale data")
print(f"   - Watch for train-test gap (overfit indicator)")

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║           DAY 1 COMPLETE: WALK-FORWARD VALIDATION                ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ Walk-forward splitter implementation                         ║
║  ✓ Rolling vs expanding window comparison                       ║
║  ✓ Visualization of splitting strategies                        ║
║  ✓ Proper time-series cross-validation                          ║
╚══════════════════════════════════════════════════════════════════╝

Tomorrow: Day 2 - Transaction Costs
""")